In [1]:
import qiime2 as q2
import pandas as pd
import numpy as np
import biom 

In [2]:
mag_wol_ft = q2.Artifact.load('/projects/thdmi/MAGs/feature-tables/woltka_biom.mag.qza') 
mag_wol_tax = pd.read_csv('/projects/thdmi/MAGs/annotations/gtdbtk-renamed/classify/classify/gtdbtk.bac120.summary.tsv', sep='\t', index_col=0)

to_keep = biom.load_table('/projects/thdmi/feature-tables/wol2_filtered_table_zebra_bmi.biom') 

In [3]:
prev_genomes = mag_wol_tax.loc[mag_wol_tax.classification.str.contains('Prevotella')]
faec_genomes = mag_wol_tax.loc[mag_wol_tax.classification.str.contains('Faecalibacterium')]
prev_taxonomy = prev_genomes['classification'].str.split(';', expand=True)
faec_taxonomy = faec_genomes['classification'].str.split(';', expand=True)

prev_taxonomy['species_name'] = np.where(prev_taxonomy[6] == 's__', 'Prevotella_MAG_' + prev_taxonomy[6].index, 
                                         prev_taxonomy[6].str.replace(' ', '_'))
faec_taxonomy['species_name'] = np.where(faec_taxonomy[6] == 's__', 'Faecalibacterium_MAG_' + faec_taxonomy[6].index, 
                                         faec_taxonomy[6].str.replace(' ', '_'))
prev_taxonomy['species_name'] = np.where(prev_taxonomy[6] == 's__Prevotella copri', 'Prevotella_copri_' + prev_taxonomy[6].index, 
                                         prev_taxonomy['species_name'].str.replace('s__', ''))
faec_taxonomy['species_name'] = np.where(faec_taxonomy['species_name'] == 's__Faecalibacterium_prausnitzii_J', 'Faecalibacterium_prausnitzii_J_' + faec_taxonomy[6].index, 
                                         faec_taxonomy['species_name'].str.replace('s__', ''))
faec_taxonomy['species_name'] = np.where(faec_taxonomy['species_name'] == 'Faecalibacterium_prausnitzii', 'Faecalibacterium_prausnitzii_' + faec_taxonomy[6].index, 
                                         faec_taxonomy['species_name'])

prev_taxonomy_key = prev_taxonomy['species_name'].to_dict()
faec_taxonomy_key = faec_taxonomy['species_name'].to_dict()

In [4]:
mag_ft_df = mag_wol_ft.view(pd.DataFrame)
mag_ft_df = mag_ft_df.loc[mag_ft_df.index.intersection(to_keep.ids())]
prev_mag_ft = mag_ft_df[mag_ft_df.columns.intersection(prev_genomes.index)]
prev_mag_ft.rename(columns=prev_taxonomy_key, inplace=True)

faec_mag_ft = mag_ft_df[mag_ft_df.columns.intersection(faec_genomes.index)]
faec_mag_ft.rename(columns=faec_taxonomy_key, inplace=True)

prev_mag_lr = prev_mag_ft.copy()
faec_mag_lr = faec_mag_ft.copy()

row_sum = prev_mag_ft.sum(axis=1)
row_sum_faec = faec_mag_lr.sum(axis=1)

for c in prev_mag_lr.columns: 
    prev_mag_lr[c] = np.log((prev_mag_lr[c] + 1) / row_sum)

for c in faec_mag_lr.columns: 
    faec_mag_lr[c] = np.log((faec_mag_lr[c] + 1) / row_sum_faec)

/tmp/ipykernel_193804/1107671229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_mag_ft.rename(columns=prev_taxonomy_key, inplace=True)
/tmp/ipykernel_193804/1107671229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faec_mag_ft.rename(columns=faec_taxonomy_key, inplace=True)


In [6]:
log_ratios = pd.read_csv("log_ratio_foods_analysis.csv", index_col=0)
main_diet_vars = pd.read_csv('main_diet_variables.txt', sep='\t')

In [7]:
metadata = pd.read_csv('/projects/thdmi/metadata/consolidated_metadata_subset.tsv', sep='\t', index_col=0)

/tmp/ipykernel_193804/2904471605.py:1: DtypeWarning: Columns (1511) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/projects/thdmi/metadata/consolidated_metadata_subset.tsv', sep='\t', index_col=0)


In [8]:
prev_mag_sex = prev_mag_lr.merge(metadata[['sex', 'thdmi_cohort']], right_index=True, left_index=True).groupby(['thdmi_cohort', 'sex']).mean()
prev_mag_sex.index = ['_'.join(i) for i in prev_mag_sex.index]

In [9]:
pd.DataFrame(data={'cols':prev_mag_ft.columns}).to_csv('prevotella_cols.csv')

In [12]:
for_lmer = log_ratios[log_ratios.columns.intersection(main_diet_vars.variable_name).union(['thdmi_cohort', 'covid_level_of_wellbeing_coded', 'bmi_cat_coded', 
                                                                                           'antibiotic_history_coded', 'log_Prevotella_to_Bacteroides', 
                                                                                           'Amount_Energy_in_kcal', 'sex'])]
full_data = for_lmer.merge(prev_mag_lr, right_index=True, left_index=True)
full_data['diet_type_coded'].replace({'Non-Vegetarian': 0, 'Vegetarian': 1}, inplace=True)
full_data['specialized_diet_exclude_dairy'].replace({False: 0, True: 1}, inplace=True)
full_data['specialized_diet_exclude_refined_sugars'].replace({False: 0, True: 1}, inplace=True)
full_data.to_csv('prevotella_lmer.csv')

In [13]:
full_data = for_lmer.merge(faec_mag_lr, right_index=True, left_index=True)
full_data['diet_type_coded'].replace({'Non-Vegetarian': 0, 'Vegetarian': 1}, inplace=True)
full_data['specialized_diet_exclude_dairy'].replace({False: 0, True: 1}, inplace=True)
full_data['specialized_diet_exclude_refined_sugars'].replace({False: 0, True: 1}, inplace=True)
full_data.to_csv('faecalibacterium_lmer.csv')

In [14]:
prev_high = full_data.loc[full_data['log_Prevotella_to_Bacteroides'] >= 0]
prev_low = full_data.loc[full_data['log_Prevotella_to_Bacteroides'] < 0]

prev_mx = full_data.loc[full_data['thdmi_cohort'] == 'Mexico']
prev_uk = full_data.loc[full_data['thdmi_cohort'] == 'UK']
prev_us = full_data.loc[full_data['thdmi_cohort'] == 'US']

In [15]:
prev_high.to_csv('prevotella_high.csv') 
prev_low.to_csv('lprevotella_low.csv') 
prev_mx.to_csv('prevotella_mexico.csv') 
prev_uk.to_csv('prevotella_uk.csv') 
prev_us.to_csv('prevotella_us.csv') 